# Download flickr dataset

In [1]:
!wget https://github.com/xu-shitong/flickr8k-CLIP-freature/raw/master/image_all_final.pickle.zip
!wget https://github.com/xu-shitong/flickr8k-CLIP-freature/raw/master/text_all_final.pickle.zip
!wget https://github.com/xu-shitong/flickr8k-CLIP-freature/raw/master/captions.txt.zip
!unzip -q image_all_final.pickle.zip 
!unzip -q text_all_final.pickle.zip
!unzip -q captions.txt.zip
!pip install transformers
!pip install gputil
!pip install psutil
!pip install humanize
!touch summary.txt

# Imports

In [1]:
from PIL import Image
import pandas as pd
import copy
from torchvision.datasets import CocoCaptions
from transformers import (
  DistilBertTokenizer, DistilBertForMaskedLM, DistilBertConfig,
  CLIPProcessor, CLIPModel as CLIP, CLIPConfig,
  activations, PreTrainedTokenizer
)
import torch
from torch.utils.data import DataLoader
from torch import nn, optim
import tqdm
import matplotlib.pyplot as plt
import os
import time
from spacy.lang.en import English
from collections import Counter
import itertools

if torch.cuda.is_available():
  dev = "cuda:0"
else:
  dev = "cpu"
device = torch.device(dev)
print("using device: ", dev)


/Users/xushitong/miniconda3/envs/mlenv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


using device:  cpu


In [ ]:
# Import packages
import os,sys,humanize,psutil,GPUtil

# Define function
def mem_report():
  print("CPU RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ))
  
  GPUs = GPUtil.getGPUs()
  for i, gpu in enumerate(GPUs):
    print('GPU {:d} ... Mem Free: {:.0f}MB / {:.0f}MB | Utilization {:3.0f}%'.format(i, gpu.memoryFree, gpu.memoryTotal, gpu.memoryUtil*100))

mem_report()

In [ ]:
# download pretrained model and tokenizer
def save_model_tokenizer(tokenizer_class, model_class, name):
  if tokenizer_class is not None:
    tokenizer = tokenizer_class.from_pretrained(name)
    tokenizer.save_pretrained(f"./tokenizers/{name}-local")
  if model_class is not None:
    model = model_class.from_pretrained(name)
    model.save_pretrained(f"./models/{name}-local/")

# save_model_tokenizer(CLIPProcessor, CLIP, "openai/clip-vit-base-patch32")
save_model_tokenizer(DistilBertTokenizer, DistilBertForMaskedLM, "distilbert-base-uncased")


# Hyperparameters

In [8]:
def series_sum_batch_average(x_hat, x):
  return (x_hat - x).abs().sum(dim=1).mean()

# hyperparameters
IN_CHANNEL = 768 # output dimention of embedding layer
BATCH_SIZE = 8
MAX_LENGTH = 16 # max text length
LEARNING_RATE = 5e-5
EPOCH_NUM = 5
ROUNDING_WEIGHT = 0.3 # weight of rounding term, the probability of regenerated sequence 
# LOSS_FUNC = nn.functional.l1_loss
LOSS_FUNC = series_sum_batch_average # loss function used between embedding 
CLIP_ADDING_METHOD = "concat" "add" # method CLIP feature is added to sequence of word embedding

# diffusion hyperparameter
BETA_MIN = 0.0001
BETA_MAX = 0.02
STEP_TOT = 1000 # total noise adding steps
COSIN_SCHEDULE = True # if alpha sequence is scheduled in cosin instead of linear patten
SAMPLE_SIZE = 100 # number of sample steps in each diffuse sequence
X_0_PREDICTION = True # if model predicts x_0 or x_{t-1}

# Define Dataset

In [3]:
# TODO: use self defined tokenizer

captions = pd.read_csv("captions.txt")["caption"]
nlp = English()

sentence_lst = []

for sentences in captions:
  word_lst = [x.text.lower() for x in nlp.tokenizer(sentences)]
  spl = [[]]
  for x, y in itertools.groupby(word_lst, lambda z: z == '.'):
      spl[-1].extend(y)
      if x: spl.append([])
  sentence_lst.extend(spl[:-1])

counter = Counter()
for input_ids in sentence_lst:
    counter.update(input_ids)
vocab_dict = {'START': 0, 'END': 1, 'UNK':2, 'PAD':3}
for k, v in counter.items():
    if v > 10:
      vocab_dict[k] = len(vocab_dict)
VOCAB_SIZE = len(vocab_dict)


In [4]:
class Flickr8kCLIPDataset(torch.utils.data.Dataset):
  def __init__(self, captions, tokenizer) -> None:
    self.caption = captions
    self.tokenizer = tokenizer

    self.train_dataset = torch.utils.data.TensorDataset(torch.load("image_all_final.pickle").to(device), torch.load("text_all_final.pickle").to(device))

  def __len__(self):
    return len(self.caption)

  def __getitem__(self, idx):
    image_clip, text_clip = self.train_dataset[idx]
    if isinstance(self.tokenizer, PreTrainedTokenizer):
      tokens = self.tokenizer(text=self.caption.loc[idx], return_tensors="pt", padding='max_length', truncation=True, max_length=MAX_LENGTH)
    else:
      ids = [0] + [vocab_dict.get(x, vocab_dict['UNK']) for x in self.caption.loc[idx][:MAX_LENGTH-2]] + [1] 
      pad_length = max(0, MAX_LENGTH - len(ids))
      tokens = dict()
      tokens["input_ids"] = torch.tensor(ids + [vocab_dict['UNK']] * pad_length)
      tokens["attention_mask"] = torch.tensor([1] * len(ids) + [0] * pad_length)

    return {
      "image_clip": image_clip, 
      "text_clip": text_clip, 
      "input_ids": tokens["input_ids"].squeeze().to(device), 
      "attention_mask": tokens["attention_mask"].squeeze().to(device)
    }

# TODO: COCO dataset

tokenizer = DistilBertTokenizer.from_pretrained("./tokenizers/distilbert-base-uncased-local/", local_files_only=True)
# tokenizer = vocab_dict

train_dataset = Flickr8kCLIPDataset(pd.read_csv("captions.txt")["caption"], tokenizer)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=BATCH_SIZE, drop_last=True)


In [ ]:
mem_report()

# Model, trainer and loss function

In [5]:
class DistilBertModel(nn.Module):
  def __init__(self, embedding, projection, config=None) -> None:
    '''
    inputs:
      embedding: clip embedding module
      config
    '''
    super().__init__()

    self.model = DistilBertForMaskedLM(config).to(device)

    self.embedding = copy.deepcopy(embedding.requires_grad_(False))
    self.projection = copy.deepcopy(projection.requires_grad_(False))
    self.projection.bias.data = torch.zeros(self.projection.bias.data.shape, device=device).requires_grad_(False)
    
    self.model.set_input_embeddings(nn.Sequential())
    self.model.set_output_embeddings(nn.Sequential())

    self.image_linear = nn.Linear(512, 768, device=device)
    self.text_linear = nn.Linear(512, 768, device=device)

  def parameters(self):
    return list(model.model.parameters()) + list(model.image_linear.parameters()) + list(model.text_linear.parameters())
  
  def forward(self, x, image_clip, text_clip, mask):
    '''
    input:
      x: [x_t ... x_t], shape: [sample_size * batch_size, seq_len, dim]
      image_clip, text_clip shape: [sample_size * batch_size, 1, clip_dim]
      mask shape: [sample_size * batch_size, seq_len + 2]

    return 
      vocab_out, shape: [sample_size * batch_size, seq_len, vocab_size]
      feature_out, shape: [sample_size * batch_size, seq_len + 2, dim]
    '''
    sample_batch_multi, _, _ = x.shape

    assert x.shape == (sample_batch_multi, MAX_LENGTH, 768)
    assert image_clip.shape == text_clip.shape == (sample_batch_multi, 1, 512)
    assert mask.shape == (sample_batch_multi, MAX_LENGTH + 2)
    
    x = torch.hstack([x, self.image_linear(image_clip), self.text_linear(text_clip)])
    x_out = self.model(x, mask)[0]    

    assert x_out.shape == (sample_batch_multi, MAX_LENGTH + 2, 768)
    return self.projection(x_out[:, :-2, :]), x_out

class DistilBertSmallModel(nn.Module):
  def __init__(self, vocab_size, config=None) -> None:
    '''
    inputs:
      embedding: clip embedding module
      config
    '''
    super().__init__()

    self.model = DistilBertForMaskedLM(config).to(device)
    
    self.model.set_input_embeddings(nn.Sequential())
    self.model.set_output_embeddings(nn.Sequential())

    self.embedding = nn.Embedding(vocab_size, IN_CHANNEL, device=device).requires_grad_(True)
    self.lm_head = nn.Linear(IN_CHANNEL, vocab_size, bias=False, device=device).requires_grad_(True)

    self.input_projection = nn.Linear(IN_CHANNEL, 768, device=device).requires_grad_(True)
    self.output_projection = nn.Linear(768, IN_CHANNEL, device=device).requires_grad_(True)

    self.image_linear = nn.Linear(512, 768, device=device).requires_grad_(True)
    self.text_linear = nn.Linear(512, 768, device=device).requires_grad_(True)

  def parameters(self):
    return list(self.model.parameters()) \
        + list(self.embedding.parameters()) \
        + list(self.lm_head.parameters()) \
        + list(self.input_projection.parameters()) \
        + list(self.output_projection.parameters()) \
        + list(self.image_linear.parameters()) \
        + list(self.text_linear.parameters())
  
  def forward(self, x, image_clip, text_clip, mask):
    '''
    input:
      x: [x_t ... x_t], shape: [sample_size * batch_size, seq_len, IN_CHANNEL]
      image_clip, text_clip shape: [sample_size * batch_size, 1, clip_dim]
      mask shape: [sample_size * batch_size, seq_len + 2]

    return 
      vocab_out, shape: [sample_size * batch_size, seq_len, vocab_size]
      feature_out, shape: [sample_size * batch_size, seq_len + 2, IN_CHANNEL]
    '''
    sample_batch_multi, _, _ = x.shape

    assert x.shape == (sample_batch_multi, MAX_LENGTH, IN_CHANNEL)
    assert image_clip.shape == text_clip.shape == (sample_batch_multi, 1, 512)
    assert mask.shape == (sample_batch_multi, MAX_LENGTH + 2)
    
    x = torch.hstack([self.input_projection(x), self.image_linear(image_clip), self.text_linear(text_clip)])
    x = self.model(x, mask)[0]
    x_out = self.output_projection(x)

    assert x_out.shape == (sample_batch_multi, MAX_LENGTH + 2, IN_CHANNEL)
    return self.lm_head(x_out[:, :-2, :]), x_out
    
origin = DistilBertForMaskedLM.from_pretrained("./models/distilbert-base-uncased-local", local_files_only=True).to(device)
configuration = DistilBertConfig()
model = DistilBertModel(origin.get_input_embeddings(), origin.get_output_embeddings(), config=configuration)

# configuration = DistilBertConfig()
# model = DistilBertSmallModel(VOCAB_SIZE, config=configuration)

# parameter only include model, no embedding layer
# trainer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
trainer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)


In [6]:
if COSIN_SCHEDULE:
  def scheduler(t):
    s = 0.008 # smalle value prevent beta_t too small, from Improved DDPM paper
    return torch.cos(torch.pi / 2 * (t/STEP_TOT + s) / (1 + s)) ** 2
  ts = torch.arange(STEP_TOT).to(device)
  alpha_cumprod = scheduler(ts) / scheduler(torch.zeros(1, device=device))
else:
  betas = torch.hstack([torch.zeros(1), torch.linspace(BETA_MIN, BETA_MAX, STEP_TOT)]).to(device)
  alphas = 1 - betas
  alpha_cumprod = torch.cumprod(alphas[:-1], 0)
def diffuse_t(x, t):
  '''
  input:
    x_shape: [batch_size, seq_len, IN_CHANNEL]
    t shape: [sample num] 
      NOTE: not necessary have hyperparameter sample_size number of element, to allow single diffuse generation

  return shape [sample_num * batch_size, seq_len, IN_CHANNEL]
  '''
  batch_size, seq_len, _ = x.shape
  sample_shape = (t.numel(), *(1, ) * len(x.shape))

  noise = torch.normal(0, 1, x.shape).to(device)
  mean = torch.sqrt(alpha_cumprod[t].reshape(sample_shape)) * x 
  epsilon = noise * torch.sqrt(1 - alpha_cumprod[t]).reshape(sample_shape)
  return (mean + epsilon).reshape((t.numel() * batch_size, seq_len, IN_CHANNEL))

def generate_diffuse_pair(x_0, t, t_next=None):
  '''
  input:
    x_0 shape: [batch_size, seq_len, IN_CHANNEL],
    t shape: [sample_num] 
      NOTE: not necessary have hyperparameter sample_size number of element, to allow single diffuse generation
  
  return (net input, net target)
    net input shape: [sample_num * batch_size, seq_len, IN_CHANNEL]
    net target shape: if t_next is None then [batch_size, seq_len, IN_CHANNEL] else [sample_num * batch_size, seq_len, IN_CHANNEL]
  '''
  if X_0_PREDICTION:
    # predict x_0
    return (diffuse_t(x_0, t), x_0)

  # predict x_{t_next}
  return (diffuse_t(x_0, t), diffuse_t(x_0, t_next))

def loss(model, x_t, x_1, x_tgt, x_0, image_clip, text_clip, mask, idx, loss_func):
  ''' 
  input: 
    model, 
    x_t, x_tgt shape: [sample_num * batch_size, seq_len, IN_CHANNEL]
      NOTE: x_tgt only used when X_0_PREDICTION is False
    x_1, x_0 shape: [batch_size, seq_len, IN_CHANNEL]
    image_clip, text_clip shape: [batch_size, clip_dim]
    mask shape: [batch_size, seq_len]
    idx shape: [batch_size, seq_len]
    loss_func

  return triple loss terms
  '''
  assert x_t.shape == (SAMPLE_SIZE * BATCH_SIZE, MAX_LENGTH, IN_CHANNEL)
  assert x_1.shape == x_0.shape == (BATCH_SIZE, MAX_LENGTH, IN_CHANNEL)
  assert image_clip.shape == text_clip.shape == (BATCH_SIZE, 512)
  assert mask.shape == (BATCH_SIZE, MAX_LENGTH)
  assert idx.shape == (BATCH_SIZE, MAX_LENGTH)

  def concat_mask(mask, append, repeat_num):
    '''
    input: 
      mask shape: []
      append shape: []
      repeat_num: scalar, how many times mask is repeated, appended shape is repeated (repeat_num * mask.shape[0]) times

    return:
      mask concat append repeated repeat_num times
    '''
    concated_mask = mask.repeat((repeat_num, 1))
    concated_mask = torch.hstack([concated_mask, append.repeat((concated_mask.shape[0], 1))])
    return concated_mask

  
  repeat_shape = (SAMPLE_SIZE, *(1, ) * (len(x_t.shape) - 1))
  image_clip = image_clip.unsqueeze(1) # shape [ batch_size, 1, clip_dim]
  text_clip = text_clip.unsqueeze(1) # shape same as above
  if CLIP_ADDING_METHOD == "concat":
    x_t_mask = concat_mask(mask, torch.tensor([1, 0], device=device), SAMPLE_SIZE)
    x_1_mask = concat_mask(mask, torch.tensor([1, 0], device=device), 1)
  elif CLIP_ADDING_METHOD == "add":
    x_t_mask = mask.repeat((SAMPLE_SIZE, 1))
    x_1_mask = mask
    x_t = 

  # x_t restore loss
  x_t_prob, x_t_hidden = model(x_t, image_clip.repeat(repeat_shape), text_clip.repeat(repeat_shape), x_t_mask)
  if X_0_PREDICTION:
    x_t_loss = loss_func(x_t_hidden[:, :-2, :], x_0.repeat(repeat_shape))
  else:
    assert x_tgt.shape == x_t.shape
    x_t_loss = loss_func(x_t_hidden[:, :-2, :], x_tgt)

  # x_1 restore loss
  x_1_prob, x_1_hidden = model(x_1, image_clip, text_clip, x_1_mask)
  x_1_loss = loss_func(x_1_hidden[:, :-2, :], x_0)

  # output sequence probability loss, applied to both x_1 and x_t restore
  idx = idx.unsqueeze(dim=-1)
  x_t_prob_loss = -(nn.functional.softmax(x_t_prob, dim=-1)).gather(-1, idx.repeat(repeat_shape)).log().sum(dim=1).mean()
  x_1_prob_loss = -(nn.functional.softmax(x_1_prob, dim=-1)).gather(-1, idx).log().sum(dim=1).mean()
  
  return x_t_loss, x_1_loss, ROUNDING_WEIGHT * (x_t_prob_loss + x_1_prob_loss)


In [12]:
CLIP_ADDING_METHOD == "add"

In [ ]:
mem_report()

# Training

In [11]:

# training TODO: forward new model
# model = torch.load("model-200-sample-pretrain-embedding.pickle")
# model.model.add_module("activation", activations.GELUActivation())
# trainer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
summary = open("temp.txt", "a")

model.train()
print("start training")
for epoch in range(EPOCH_NUM):
  acc_x_t = 0
  acc_x_1 = 0
  acc_prob = 0
  with tqdm.tqdm(train_loader, unit="batch") as tepoch: 
    for batch_num, x in enumerate(tepoch):
  # for batch_num, x in enumerate(train_loader):

      x_0 = model.embedding(x["input_ids"])
      repeat_shape = (SAMPLE_SIZE, *(1, ) * (len(x_0.shape) - 1))
      t = torch.randint(0, STEP_TOT, repeat_shape, device=device)
      
      if X_0_PREDICTION:
        x_t = diffuse_t(x_0, t)
        x_tgt = None
      else:
        x_t, x_tgt = generate_diffuse_pair(x_0, t, torch.max(t - 30, torch.zeros(t.shape, device=device, dtype=torch.int64)))
      x_1 = diffuse_t(x_0, torch.ones(1, dtype=torch.int64, device=device))

      trainer.zero_grad()
      x_t_loss, x_1_loss, prob_loss = loss(
        model, 
        x_t, x_1, x_tgt, x_0, 
        x["image_clip"], x["text_clip"], 
        x["attention_mask"], 
        x["input_ids"], 
        LOSS_FUNC
      )
      
      l = x_t_loss + x_1_loss + prob_loss
      l.backward()
      trainer.step()
      
      acc_x_t += x_t_loss 
      acc_x_1 += x_1_loss 
      acc_prob += prob_loss

      tepoch.set_description(f"batch {batch_num}")
      tepoch.set_postfix(
                         x_t_hidden=x_t_loss.item(),
                         x_1_loss=x_1_loss.item(),
                         prob_loss=prob_loss.item(),
                         tot_loss=l.item())

      if batch_num % 25 == 0:
        summary.write(f"batch {batch_num} avg x_t_loss, x_1_loss, prob_loss: {acc_x_t / (batch_num + 1)}, {acc_x_1 / (batch_num + 1)}, {acc_prob / (batch_num + 1)}\n")
      break

  print(f"epoch {epoch} average x_t_loss, x_1_loss, prob_loss: {acc_x_t / len(train_loader)}, {acc_x_1 / len(train_loader)}, {acc_prob / len(train_loader)}")
  break
summary.close()


start training


sample 0:   0%|          | 0/10113 [01:49<?, ?batch/s, prob_loss=8.66, tot_loss=12.9, x_1_loss=0.966, x_t_hidden=3.27]


NameError: name 'sample_num' is not defined

# Evaluate

In [9]:
# trial on inference
model = torch.load("model-200-sample-pretrain-embedding.pickle")
model.model.add_module("activation", activations.GELUActivation())
model.eval()
idx = 11
origin_text = train_dataset.caption.loc[idx]
print("origin text: ", origin_text)

sample = train_dataset[idx]
image_clip = sample["image_clip"][None, None, :]
text_clip = sample["text_clip"][None, None, :]
x_0 = model.embedding(sample["input_ids"].unsqueeze(0))
t = 500
print(f"t = {t}")
x_t = diffuse_t(x_0, torch.tensor([t], dtype=torch.int64, device=device))
mask = sample["attention_mask"].unsqueeze(0)

none_mask = torch.hstack([mask, torch.ones((mask.shape[0], 2), device=device)])
text_mask = torch.hstack([mask, torch.tensor([1, 0], device=device).repeat((mask.shape[0], 1))])
all_mask = torch.hstack([mask, torch.zeros((mask.shape[0], 2), device=device)])
applied_mask = text_mask

# multi-step inference
restored = x_t
for i in range(5):
  out, restored = model(restored[:, :MAX_LENGTH, :], image_clip, text_clip, applied_mask)
  print("inferred: ", train_dataset.tokenizer.decode(out.argmax(dim=-1)[0])[:len(origin_text) + 10])

# effectiveness of model on large t
print("text t effectiveness")
for i in range(1, STEP_TOT, 100):
  x_t = diffuse_t(x_0, torch.tensor([i], dtype=torch.int64, device=device))
  out, _ = model(x_t, image_clip, text_clip, applied_mask) 

  print("t: ", i, "restore: ", train_dataset.tokenizer.decode(out.argmax(dim=-1)[0])[:len(origin_text) + 20])



origin text:  A little girl is sitting in front of a large painted rainbow .
t = 500
inferred:  [CLS] a little girl is sitting in front of a large white yellow. [SEP] [
inferred:  [CLS] a little girl is sitting in front of a large white yellow. [SEP] [
inferred:  [CLS] a little girl is sitting in front of a large white yellow. [SEP] [
inferred:  [CLS] a little girl is sitting in front of a large down yellow. [SEP] [P
inferred:  [CLS] a little girl is sitting in front of a large sand yellow. [SEP] [P
text t effectiveness
t:  1 restore:  [CLS] a little girl is sitting in front of a large over yellow. [SEP] [PAD]
t:  101 restore:  [CLS] a little girl is sitting in front of a large of yellow. [SEP] [PAD]
t:  201 restore:  [CLS] a little girl is sitting in front of a large over yellow. [SEP] [PAD]
t:  301 restore:  [CLS] a little girl is sitting in front of a large at yellow. [SEP] [PAD]
t:  401 restore:  [CLS] a little girl is sitting in front of a large on rides. [SEP] [PAD]
t:  501 resto

In [ ]:
text = ["a photo of a cat", "a photo of a warship", "a photo of a boy", "a photo of a girl"]
image = Image.open("Shropshire.jpeg")
# plt.imshow(image)
# plt.show()
inputs = clip_processor(text=text, images=image, return_tensors="pt", padding=True)
inputs_no = clip_processor(text=text, images=None, return_tensors="pt", padding=True)

print(inputs.keys(), torch.all(inputs.input_ids == inputs_no.input_ids))

outputs = clip.get_text_features(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
# outputs = clip(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], pixel_values=torch.zeros((3,1,1)))

# outputs = clip(**inputs)
print(outputs.text_embeds.shape, outputs.image_embeds.shape, 
outputs.text_model_output["last_hidden_state"].shape, outputs.text_model_output["pooler_output"].shape, 
outputs.vision_model_output["last_hidden_state"].shape, outputs.vision_model_output["pooler_output"].shape, 
)
outputs.text_embeds, outputs.image_embeds, outputs.text_model_output, outputs.vision_model_output
# logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
# probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities

# print(probs, text[probs.argmax(dim=-1)[0].item()])

In [ ]:
torch.save(model.cpu(), "model.pickle")